In [1]:
!pip install langchain_community
!pip install pymupdf
!pip install -qU pypdf
!pip install -qU langchain-text-splitters
!pip install -qU langchain-text-splitters FlagEmbedding faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.3.44
    Uninstalling langsmith-0.3.44:
      Successfully uninstalled langsmith-0.3.44
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.63
    Uninstalling langchain-core-0.3.63:
      Successfully uninstalled langchain-core-0.3.63
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.2/304.2 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.9/163.9 kB 5.7 MB/s eta 0:00:00
  Preparing

In [2]:
!python -m pip show FlagEmbedding

Name: FlagEmbedding
Version: 1.3.5
Summary: FlagEmbedding
Home-page: https://github.com/FlagOpen/FlagEmbedding
Author: 
Author-email: 2906698981@qq.com
License: 
Location: /usr/local/lib/python3.11/dist-packages
Requires: accelerate, datasets, ir-datasets, peft, protobuf, sentence-transformers, sentencepiece, torch, transformers
Required-by: 


In [3]:
!python -m pip install -U "FlagEmbedding>=1.2.7"

In [4]:
from langchain_community.vectorstores import FAISS

In [ ]:
# ollama 설치
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
# 버전 확인
!ollama --version

In [ ]:
# ollama 서버 백그라운드에서 실행
!nohup ollama serve > log.txt 2>&1 &

In [7]:
# !ollama pull llama3

In [ ]:
# 모델 다운로드
!ollama pull exaone3.5:2.4b

In [ ]:
# ollama 서버에서 사용 가능한 모델 확인
!curl http://localhost:11434/api/tags

{"models":[{"name":"exaone3.5:2.4b","model":"exaone3.5:2.4b","modified_at":"2025-06-15T16:04:54.908644825Z","size":1644933401,"digest":"13644fc3d28eaaff502f903406496cad15df31e367cb7be988c132def104ba42","details":{"parent_model":"","format":"gguf","family":"exaone","families":["exaone"],"parameter_size":"2.7B","quantization_level":"Q4_K_M"}}]}

In [14]:
# 1) 경로·파라미터 정의(완료)

PDF_PATH   = r"/content/drive/MyDrive/Colab Notebooks/pdf/2022개정교육과정자유학기운영안내서.pdf"   # ← 처리할 PDF 경로
INDEX_DIR  = "faiss_bge_m3_index"                       # 인덱스 저장 폴더
CHUNK_SIZE = 1500                                       # 청크 길이(문자)
CHUNK_OVER = 250

In [21]:
# 파일 추가(완료)

PDF_PATH = r"/content/drive/MyDrive/Colab Notebooks/pdf/2025년학생안전종합계획(발송용).pdf"   # ← 처리할 PDF 경로
INDEX_DIR  = "faiss_bge_m3_index"                       # 인덱스 저장 폴더
CHUNK_SIZE = 1500                                       # 청크 길이(문자)
CHUNK_OVER = 250

In [24]:
# 파일 추가(완료)

PDF_PATH = r"/content/drive/MyDrive/Colab Notebooks/pdf/2025자유학기운영참고자료.pdf"   # ← 처리할 PDF 경로
INDEX_DIR  = "faiss_bge_m3_index"                       # 인덱스 저장 폴더
CHUNK_SIZE = 1500                                       # 청크 길이(문자)
CHUNK_OVER = 250

In [ ]:
# 파일 추가

PDF_PATH = r"/content/2025중등교원인사업무처리요령1권.pdf"   # ← 처리할 PDF 경로
INDEX_DIR  = "faiss_bge_m3_index"                       # 인덱스 저장 폴더
CHUNK_SIZE = 1500                                       # 청크 길이(문자)
CHUNK_OVER = 250

In [15]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
🔹 BGE-M3 + FAISS 누적 인덱싱 스크립트
   · PDF 하나씩 처리할 때마다 같은 인덱스 폴더 안으로 계속 벡터를 추가합니다.
   · 인덱스 폴더가 없으면 새로 만들고, 있으면 자동으로 로드하여 add_documents().
   · 임베딩-LLM 일체 API 호출 없이 전부 로컬에서 구동됩니다.
"""

# 0) 필수 패키지 (첫 실행 시 한 번만 설치)
#    ── 주피터 노트북이면 !pip 로, 로컬 쉘이면 pip install …
!pip install -qU langchain-community langchain-text-splitters \
                faiss-cpu FlagEmbedding pymupdf


# 2) 모듈 import
import numpy as np
from pathlib import Path
from FlagEmbedding import FlagModel
from langchain.embeddings.base import Embeddings
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

# 3) BGE-M3 임베딩 래퍼 (LangChain 호환)
class BGEEmbedding(Embeddings):
    """FlagEmbedding → LangChain 래퍼 (float32 + L2 정규화)"""

    def __init__(self, model_name="BAAI/bge-m3", fp16=True):
        self.model = FlagModel(model_name, use_fp16=fp16)

    def _encode(self, texts):
        vecs = self.model.encode(texts, batch_size=32, max_length=8192)
        vecs = np.asarray(vecs, dtype="float32")
        vecs /= np.linalg.norm(vecs, axis=1, keepdims=True) + 1e-12
        return vecs

    def embed_documents(self, texts):
        return self._encode(texts)

    def embed_query(self, text):
        return self._encode([text])[0]

emb_wrapper = BGEEmbedding(fp16=True)  # GPU 없으면 fp16=False

# 4) PDF → 문서 로드 & 청크 분할
loader = PyMuPDFLoader(PDF_PATH)
docs = loader.load()  # 페이지 단위 LangChain Document 리스트

splitter = RecursiveCharacterTextSplitter(
               chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVER
           )
chunks = splitter.split_documents(docs)
print(f"📄  {PDF_PATH} → 분할 청크 {len(chunks):,}개")


# 5) FAISS 인덱스: 로드 후 add  ⟷  최초 생성
if Path(INDEX_DIR).exists():
    print("🔄 기존 인덱스 로드 중…")
    db = FAISS.load_local(
        INDEX_DIR,
        emb_wrapper,
        allow_dangerous_deserialization=True
    )
    print(f"현재 벡터 수: {db.index.ntotal:,}")

    print("새 문서를 인덱스에 추가 중…")
    db.add_documents(chunks)
else:
    print("인덱스 최초 생성 중…")
    db = FAISS.from_documents(chunks, emb_wrapper)

# 저장
db.save_local(INDEX_DIR)
print(f"저장 완료!  총 벡터 {db.index.ntotal:,}개 → {INDEX_DIR}/")

📄  /content/drive/MyDrive/Colab Notebooks/pdf/2022개정교육과정자유학기운영안내서.pdf → 분할 청크 149개
인덱스 최초 생성 중…


pre tokenize: 100%|██████████| 5/5 [00:00<00:00, 39.69it/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Inference Embeddings: 100%|██████████| 5/5 [00:02<00:00,  1.96it/s]


저장 완료!  총 벡터 149개 → faiss_bge_m3_index/


In [16]:
if __name__ == "__main__":
    query = "학교안전사고 예방 ·관리 체계의 고도화 과제에 대한 담당부서는 어디고 연락처가 어떻게 돼?"
    hits  = db.similarity_search(query, k=3)
    print("\nTop-3 유사 청크")
    for d in hits:
        pg = d.metadata.get("page_number", "?")
        print(f"(p.{pg}) {d.page_content[:120]} …")


Top-3 유사 청크
(p.?) 139
부록  디지털 도구 목록
대분류중분류소분류
에듀테크
소개
가격
대상
관
리
학
급
운
영
일
정
관
리
노션
문서 작성, 업무 계획, 업무 협업 등을 할 수 있는 
서비스이며 페이지 구성이 유연하여 쉽게 활용 …
(p.?) 자유학기 
편성·운영 방법 
자유학기 수업 개선
자유학기 평가 운영
학습자 맞춤 학습지원
진로탐색 활동 내실화
학교 준비 사항
자유학기 운영 Q&A
20
24
27
35
41
46 …
(p.?) 8
2022 개정 교육과정 자유학기 운영 안내서
운영 중점
5
디지털 기반의 자유학기 학생 참여형 수업 및 과정 중심 평가 개선
책임교육학년 운영 및 기초소양 함양을 통한 학생 맞춤형 학습지원
자유학기-진로연계교육  …


In [35]:
!pip install -U bitsandbytes  accelerate

In [17]:
import requests

# Ollama 서버 주소 (기본값은 localhost:11434)
OLLAMA_API_URL = "http://localhost:11434/api/generate"

# 사용자 질문
query = "2022 개정 교육과정에서 자유학기 활동은 어떤 두 가지 영역으로 조정되었나요?"

# 요청 payload
payload = {
    "model": "exaone3.5:2.4b",  # 사용 모델 이름
    "prompt": query,
    "stream": False             # 전체 응답을 한 번에 받기 (True면 스트리밍 응답)
}

# POST 요청으로 응답 받기
response = requests.post(OLLAMA_API_URL, json=payload)

# 결과 출력
if response.ok:
    result = response.json()
    print("🧠 답변:", result.get("response", "No response"))
else:
    print("❌ 오류 발생:", response.status_code, response.text)

🧠 답변: 2022 개정 교육과정에서 자유학기제는 **융합형 학습 및 창의적 체험 활동** 두 가지 주요 영역으로 재조정되었습니다. 

이 변화는 학생들이 실질적인 문제 해결 능력과 창의적 사고력을 키우는 데 초점을 맞추고자 함을 반영합니다. 구체적으로:

1. **융합형 학습**: 여러 교과목 간의 연계를 강화하여 학생들이 다양한 분야 간의 교차 학습을 경험할 수 있도록 했습니다. 예를 들어, 과학과 예술, 기술과 인문학 등을 결합한 프로젝트를 통해 통합적인 사고력을 키울 수 있게 되었습니다.

2. **창의적 체험 활동**: 학생들의 흥미와 적성에 맞춘 다양한 체험 학습 기회를 확대했습니다. 이는 개인의 잠재력을 발굴하고 창의적인 접근 방식을 촉진하는 데 중점을 두고 있습니다. 예를 들어, 봉사활동, 동아리 활동, 독립 연구 등이 포함됩니다.

이러한 조정은 학생 중심의 교육 환경을 조성하고 미래 사회에서 요구되는 역량을 향상시키는 데 목표를 두고 있습니다.
